In [100]:
from pymongo import MongoClient
import sys
sys.path.append('../statistical_calculations')  # Adds the statistical_calculations directory to the path
import disproportionaly_analysis
from tqdm import tqdm
import json
from datetime import datetime, timedelta 
from calendar import monthrange

# Connect to your MongoDB instance
client = MongoClient('mongodb://localhost:27017/')
db = client['vaers']  
col = db['reports']

In [101]:
end_date = datetime(2020, 11, 30)

# Define a function to reduce the date range by a month
def reduce_date_range_by_month(start_date):
    # Get the year and month of the start date
    year = start_date.year
    month = start_date.month
    
    # Calculate the new year and month
    if month == 1:  # If the current month is January
        new_year = year - 1
        new_month = 12  # December of the previous year
    else:
        new_year = year
        new_month = month - 1
    
    # Construct the new start date
    _, last_day_of_previous_month = monthrange(new_year, new_month)
    new_start_date = datetime(new_year, new_month, last_day_of_previous_month)
    
    return new_start_date

def reduce_date_range_by_day(start_date):
    new_start_date = start_date - timedelta(days=1)  # Subtract one day from the start_date
    return new_start_date

### PFIZER/BIONTECH

In [143]:
file_name_filtered = "Signals_Pfizer.json"
with open(file_name_filtered, 'r') as f:
    Signals_Pfizer = json.load(f)

In [144]:
filtered_signals_Pfizer = [signal for signal in Signals_Pfizer if signal["status"] != "Not found"]

In [145]:
len(filtered_signals_Pfizer)

34

In [146]:
from datetime import datetime

# Loop until a signal is not found anymore
signals_dict_Pfizer = []
i= 0
for reaction in tqdm(filtered_signals_Pfizer):
    i+=1
    symptom = reaction['symptom']
    start_date = datetime(2023, 12, 31)
    first_found = datetime(2000, 1, 1)
    while True:
        N_query = {"RECVDATE": {"$lte": start_date}}
        N = col.count_documents(N_query)
        
        D_query = {
            "RECVDATE": {"$lte": start_date},
            "vax_data": {
                "$elemMatch": {
                    "VAX_MANU": reaction["manufacturer"],
                    "VAX_TYPE": reaction["vaccine"]
                }
            }
        }
        D = col.count_documents(D_query)
        
        E_query = {
            "RECVDATE": {"$lte": start_date},
            "symptoms": {
                "$elemMatch": {
                    "$in": [reaction["symptom"]]
                }
            }
        }
        E = col.count_documents(E_query)
        
        DE_query = {'$and': [D_query, E_query]}
        DE = col.count_documents(DE_query)
        if DE >= 3: 
        
            De = D - DE
            dE = E - DE
            de = N - (DE + De + dE)
            
                
            # Ensure all necessary data is available
            if None in [DE, dE, De, de]:
                continue
            
            contingency_table = [
                [DE, dE],
                [De, de]
            ]
            #print(contingency_table)
            # Compute statistical measures
            rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
            rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
            
            prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
            prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
            
            ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
            ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
            
            chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
            IC = disproportionaly_analysis.information_component(contingency_table)
            IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
            
            # Determine if the reaction meets criteria for a signal
            method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
            method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                
            if method_1 or method_2:
                #print(f"Signal found for {start_date}")
                first_found = start_date
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")
            else:
                #print("Signal not found")
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
            if start_date < end_date:
                break
        else:
            break
            
    if first_found == datetime(2000, 1, 1):
        print(f"Saved {symptom} as first_found = None because it was not detected by the end of an month")
        signals_dict_Pfizer.append({"manufacturer": reaction["manufacturer"], "symptom": reaction["symptom"], "first_found": "NaN"})
    else:
            
        print(f"found month {first_found} for symptom {i} {symptom}, checking day")
        start_date = first_found
        while True:
            #print(f"checking day {start_date}")
                    #print(f"checking symptom {reaction['symptom']}")
            N_query = {"RECVDATE": {"$lte": start_date}}
            N = col.count_documents(N_query)
            
            D_query = {
                "RECVDATE": {"$lte": start_date},
                "vax_data": {
                    "$elemMatch": {
                        "VAX_MANU": reaction["manufacturer"],
                        "VAX_TYPE": reaction["vaccine"]
                    }
                }
            }
            D = col.count_documents(D_query)
            
            E_query = {
                "RECVDATE": {"$lte": start_date},
                "symptoms": {
                    "$elemMatch": {
                        "$in": [reaction["symptom"]]
                    }
                }
            }
            E = col.count_documents(E_query)
            
            DE_query = {'$and': [D_query, E_query]}
            DE = col.count_documents(DE_query)
            if DE >= 3: 
            
                De = D - DE
                dE = E - DE
                de = N - (DE + De + dE)
                
                    
                # Ensure all necessary data is available
                if None in [DE, dE, De, de]:
                    continue
                
                contingency_table = [
                    [DE, dE],
                    [De, de]
                ]
                #print(contingency_table)
                # Compute statistical measures
                rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
                rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
                
                prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
                prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
                
                ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
                ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
                
                chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
                IC = disproportionaly_analysis.information_component(contingency_table)
                IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
                
                # Determine if the reaction meets criteria for a signal
                method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
                method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                    
                if method_1 or method_2:
                    #print(f"Signal found for {start_date}")
                    first_found = start_date
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Pfizer.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop        
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")
                else:
                    #print("Signal not found")
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Pfizer.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
    
            else: 
                #print(f"DE < 3 for {start_date}")
                signal_info = {
                    "manufacturer": reaction["manufacturer"],
                    "symptom": reaction["symptom"],
                    "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                }
                signals_dict_Pfizer.append(signal_info)
                print(f"signal {i} {symptom} found at {first_found}")
                break
    

  0%|          | 0/34 [00:00<?, ?it/s]

found month 2021-01-31 00:00:00 for symptom 1 Vomiting, checking day


  3%|▎         | 1/34 [07:31<4:08:22, 451.60s/it]

signal 1 Vomiting found at 2021-01-15 00:00:00
found month 2020-12-31 00:00:00 for symptom 2 Lymphadenopathy, checking day


  6%|▌         | 2/34 [13:36<3:33:41, 400.66s/it]

...

signal 32 Malaise found at 2020-12-16 00:00:00


 97%|█████████▋| 33/34 [3:21:15<06:27, 387.78s/it]

Saved Injection site pruritus as first_found = None because it was not detected by the end of an month


100%|██████████| 34/34 [3:23:22<00:00, 358.91s/it]

Saved Extensive swelling of vaccinated limb as first_found = None because it was not detected by the end of an month


In [147]:
file_name_filtered = "First_found_filtered_Signals_Pfizer.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_dict_Pfizer, f)

In [106]:
signals_dict_Pfizer

[{'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Lymphadenopathy',
  'first_found': '2020-12-24'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Angioedema',
  'first_found': '2021-01-07'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Decreased appetite',
  'first_found': '2021-01-07'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Insomnia',
  'first_found': '2021-03-01'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Headache',
  'first_found': '2020-12-16'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Dizziness',
  'first_found': '2020-12-15'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Facial paralysis',
  'first_found': '2020-12-18'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Paraesthesia',
  'first_found': '2020-12-15'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Hypoaesthesia',
  'first_found': '2020-12-17'},
 {'manufacturer': 'PFIZER\\BIONTECH',
  'symptom': 'Myocarditis',
  'first_found': '2021-03-25'},
 {'manufac

### MODERNA

In [167]:
file_name_filtered = "Signals_Moderna.json"
with open(file_name_filtered, 'r') as f:
    Signals_Moderna = json.load(f)

In [168]:
filtered_signals_Moderna = [signal for signal in Signals_Moderna if signal["status"] != "Not found"]

In [169]:
len(filtered_signals_Moderna)

35

In [170]:
from datetime import datetime

# Loop until a signal is not found anymore
signals_dict_Moderna= []
i= 0
for reaction in tqdm(filtered_signals_Moderna):
    i+=1
    symptom = reaction['symptom']
    start_date = datetime(2023, 12, 31)
    first_found = datetime(2000, 1, 1)
    while True:
        N_query = {"RECVDATE": {"$lte": start_date}}
        N = col.count_documents(N_query)
        
        D_query = {
            "RECVDATE": {"$lte": start_date},
            "vax_data": {
                "$elemMatch": {
                    "VAX_MANU": reaction["manufacturer"],
                    "VAX_TYPE": reaction["vaccine"]
                }
            }
        }
        D = col.count_documents(D_query)
        
        E_query = {
            "RECVDATE": {"$lte": start_date},
            "symptoms": {
                "$elemMatch": {
                    "$in": [reaction["symptom"]]
                }
            }
        }
        E = col.count_documents(E_query)
        
        DE_query = {'$and': [D_query, E_query]}
        DE = col.count_documents(DE_query)
        if DE >= 3: 
        
            De = D - DE
            dE = E - DE
            de = N - (DE + De + dE)
            
                
            # Ensure all necessary data is available
            if None in [DE, dE, De, de]:
                continue
            
            contingency_table = [
                [DE, dE],
                [De, de]
            ]
            #print(contingency_table)
            # Compute statistical measures
            rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
            rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
            
            prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
            prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
            
            ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
            ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
            
            chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
            IC = disproportionaly_analysis.information_component(contingency_table)
            IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
            
            # Determine if the reaction meets criteria for a signal
            method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
            method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                
            if method_1 or method_2:
                #print(f"Signal found for {start_date}")
                first_found = start_date
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")
            else:
                #print("Signal not found")
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
            if start_date < end_date:
                break
        else:
            break
            
    if first_found == datetime(2000, 1, 1):
        print(f"Saved {symptom} as first_found = None because it was not detected by the end of an month")
        signals_dict_Moderna.append({"manufacturer": reaction["manufacturer"], "symptom": reaction["symptom"], "first_found": "NaN"})
    else:
            
        print(f"found month {first_found} for symptom {i} {symptom}, checking day")
        start_date = first_found
        while True:
            #print(f"checking day {start_date}")
                    #print(f"checking symptom {reaction['symptom']}")
            N_query = {"RECVDATE": {"$lte": start_date}}
            N = col.count_documents(N_query)
            
            D_query = {
                "RECVDATE": {"$lte": start_date},
                "vax_data": {
                    "$elemMatch": {
                        "VAX_MANU": reaction["manufacturer"],
                        "VAX_TYPE": reaction["vaccine"]
                    }
                }
            }
            D = col.count_documents(D_query)
            
            E_query = {
                "RECVDATE": {"$lte": start_date},
                "symptoms": {
                    "$elemMatch": {
                        "$in": [reaction["symptom"]]
                    }
                }
            }
            E = col.count_documents(E_query)
            
            DE_query = {'$and': [D_query, E_query]}
            DE = col.count_documents(DE_query)
            if DE >= 3: 
            
                De = D - DE
                dE = E - DE
                de = N - (DE + De + dE)
                
                    
                # Ensure all necessary data is available
                if None in [DE, dE, De, de]:
                    continue
                
                contingency_table = [
                    [DE, dE],
                    [De, de]
                ]
                #print(contingency_table)
                # Compute statistical measures
                rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
                rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
                
                prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
                prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
                
                ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
                ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
                
                chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
                IC = disproportionaly_analysis.information_component(contingency_table)
                IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
                
                # Determine if the reaction meets criteria for a signal
                method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
                method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                    
                if method_1 or method_2:
                    #print(f"Signal found for {start_date}")
                    first_found = start_date
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Moderna.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop        
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")
                else:
                    #print("Signal not found")
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Moderna.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
    
            else: 
                #print(f"DE < 3 for {start_date}")
                signal_info = {
                    "manufacturer": reaction["manufacturer"],
                    "symptom": reaction["symptom"],
                    "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                }
                signals_dict_Moderna.append(signal_info)
                print(f"signal {i} {symptom} found at {first_found}")
                break
    

  0%|          | 0/35 [00:00<?, ?it/s]

found month 2020-12-31 00:00:00 for symptom 1 Asthenia, checking day


  3%|▎         | 1/35 [03:45<2:08:03, 225.98s/it]

signal 1 Asthenia found at 2020-12-31 00:00:00
found month 2020-12-31 00:00:00 for symptom 2 Hyperhidrosis, checking day


  6%|▌         | 2/35 [08:49<2:29:30, 271.82s/it]

signal 2 Hyperhidrosis found at 2020-12-23 00:00:00
found month 2020-12-31 00:00:00 for symptom 3 Lymphadenopathy, checking day


  9%|▊         | 3/35 [13:40<2:29:36, 280.50s/it]

signal 3 Lymphadenopathy found at 2020-12-30 00:00:00
found month 2020-12-31 00:00:00 for symptom 4 Pruritus, checking day


  9%|▊         | 3/35 [18:01<3:12:16, 360.50s/it]


KeyboardInterrupt: 

In [ ]:
file_name_filtered = "First_found_filtered_Signals_Moderna.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_dict_Moderna, f)

In [ ]:
len(signals_dict_Moderna)

### JANSSEN

In [148]:
file_name_filtered = "Signals_Janssen.json"
with open(file_name_filtered, 'r') as f:
    Signals_Janssen = json.load(f)

In [149]:
filtered_signals_Janssen =  [signal for signal in Signals_Janssen if signal["status"] != "Not found"]

In [150]:
from datetime import datetime

# Loop until a signal is not found anymore
signals_dict_Janssen = []
i= 0
for reaction in tqdm(filtered_signals_Janssen):
    i+=1
    symptom = reaction['symptom']
    start_date = datetime(2023, 12, 31)
    first_found = datetime(2000, 1, 1)
    while True:
        N_query = {"RECVDATE": {"$lte": start_date}}
        N = col.count_documents(N_query)
        
        D_query = {
            "RECVDATE": {"$lte": start_date},
            "vax_data": {
                "$elemMatch": {
                    "VAX_MANU": reaction["manufacturer"],
                    "VAX_TYPE": reaction["vaccine"]
                }
            }
        }
        D = col.count_documents(D_query)
        
        E_query = {
            "RECVDATE": {"$lte": start_date},
            "symptoms": {
                "$elemMatch": {
                    "$in": [reaction["symptom"]]
                }
            }
        }
        E = col.count_documents(E_query)
        
        DE_query = {'$and': [D_query, E_query]}
        DE = col.count_documents(DE_query)
        if DE >= 3: 
        
            De = D - DE
            dE = E - DE
            de = N - (DE + De + dE)
            
                
            # Ensure all necessary data is available
            if None in [DE, dE, De, de]:
                continue
            
            contingency_table = [
                [DE, dE],
                [De, de]
            ]
            #print(contingency_table)
            # Compute statistical measures
            rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
            rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
            
            prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
            prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
            
            ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
            ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
            
            chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
            IC = disproportionaly_analysis.information_component(contingency_table)
            IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
            
            # Determine if the reaction meets criteria for a signal
            method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
            method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                
            if method_1 or method_2:
                #print(f"Signal found for {start_date}")
                first_found = start_date
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")
            else:
                #print("Signal not found")
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
            if start_date < end_date:
                break
        else:
            break
            
    if first_found == datetime(2000, 1, 1):
        print(f"Saved {symptom} as first_found = None because it was not detected by the end of an month")
        signals_dict_Janssen.append({"manufacturer": reaction["manufacturer"], "symptom": reaction["symptom"], "first_found": "NaN"})
    else:
            
        print(f"found month {first_found} for symptom {i} {symptom}, checking day")
        start_date = first_found
        while True:
            #print(f"checking day {start_date}")
                    #print(f"checking symptom {reaction['symptom']}")
            N_query = {"RECVDATE": {"$lte": start_date}}
            N = col.count_documents(N_query)
            
            D_query = {
                "RECVDATE": {"$lte": start_date},
                "vax_data": {
                    "$elemMatch": {
                        "VAX_MANU": reaction["manufacturer"],
                        "VAX_TYPE": reaction["vaccine"]
                    }
                }
            }
            D = col.count_documents(D_query)
            
        
            E_query = {
                "RECVDATE": {"$lte": start_date},
                "symptoms": {
                    "$elemMatch": {
                        "$in": [reaction["symptom"]]
                    }
                }
            }
            E = col.count_documents(E_query)
            
            DE_query = {'$and': [D_query, E_query]}
            DE = col.count_documents(DE_query)
            if DE >= 3: 
            
                De = D - DE
                dE = E - DE
                de = N - (DE + De + dE)
                
                    
                # Ensure all necessary data is available
                if None in [DE, dE, De, de]:
                    continue
                
                contingency_table = [
                    [DE, dE],
                    [De, de]
                ]
                #print(contingency_table)
                # Compute statistical measures
                rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
                rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
                
                prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
                prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
                
                ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
                ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
                
                chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
                IC = disproportionaly_analysis.information_component(contingency_table)
                IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
                
                # Determine if the reaction meets criteria for a signal
                method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
                method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                    
                if method_1 or method_2:
                    #print(f"Signal found for {start_date}")
                    first_found = start_date
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Janssen.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop        
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")
                else:
                    #print("Signal not found")
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Janssen.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
    
            else: 
                #print(f"DE < 3 for {start_date}")
                signal_info = {
                    "manufacturer": reaction["manufacturer"],
                    "symptom": reaction["symptom"],
                    "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                }
                signals_dict_Janssen.append(signal_info)
                print(f"signal {i} {symptom} found at {first_found}")
                break
    

  3%|▎         | 1/35 [01:06<37:37, 66.39s/it]

Saved Lymphadenopathy as first_found = None because it was not detected by the end of an month
found month 2021-04-30 00:00:00 for symptom 2 Immune thrombocytopenia, checking day


  6%|▌         | 2/35 [02:19<38:46, 70.50s/it]

signal 2 Immune thrombocytopenia found at 2021-04-24 00:00:00
found month 2021-03-31 00:00:00 for symptom 3 Headache, checking day


  9%|▊         | 3/35 [04:55<58:22, 109.44s/it]

...

Saved Injection site swelling as first_found = None because it was not detected by the end of an month
found month 2021-03-31 00:00:00 for symptom 34 Chills, checking day


 97%|█████████▋| 34/35 [54:31<01:48, 108.68s/it]

signal 34 Chills found at 2021-03-05 00:00:00
found month 2021-03-31 00:00:00 for symptom 35 Pyrexia, checking day


100%|██████████| 35/35 [57:14<00:00, 98.12s/it] 

signal 35 Pyrexia found at 2021-03-06 00:00:00


In [151]:
file_name_filtered = "First_found_filtered_Signals_Janssen.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_dict_Janssen, f)

In [114]:
len(signals_dict_Janssen)

23

### NOVAVAX

In [138]:
file_name_filtered = "Signals_Novavax.json"
with open(file_name_filtered, 'r') as f:
    Signals_Novavax = json.load(f)

In [139]:
filtered_signals_Novavax = [signal for signal in Signals_Novavax if signal["status"] != "Not found"]

In [140]:
len(filtered_signals_Novavax)

23

In [141]:
from datetime import datetime

# Loop until a signal is not found anymore
signals_dict_Novavax = []
i= 0
for reaction in tqdm(filtered_signals_Novavax):
    i+=1
    symptom = reaction['symptom']
    start_date = datetime(2023, 12, 31)
    first_found = datetime(2000, 1, 1)
    while True:
        N_query = {"RECVDATE": {"$lte": start_date}}
        N = col.count_documents(N_query)
        
        D_query = {
            "RECVDATE": {"$lte": start_date},
            "vax_data": {
                "$elemMatch": {
                    "VAX_MANU": reaction["manufacturer"],
                    "VAX_TYPE": reaction["vaccine"]
                }
            }
        }
        D = col.count_documents(D_query)
        
        E_query = {
            "RECVDATE": {"$lte": start_date},
            "symptoms": {
                "$elemMatch": {
                    "$in": [reaction["symptom"]]
                }
            }
        }
        E = col.count_documents(E_query)
        
        DE_query = {'$and': [D_query, E_query]}
        DE = col.count_documents(DE_query)
        if DE >= 3: 
        
            De = D - DE
            dE = E - DE
            de = N - (DE + De + dE)
            
                
            # Ensure all necessary data is available
            if None in [DE, dE, De, de]:
                continue
            
            contingency_table = [
                [DE, dE],
                [De, de]
            ]
            #print(contingency_table)
            # Compute statistical measures
            rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
            rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
            
            prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
            prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
            
            ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
            ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
            
            chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
            IC = disproportionaly_analysis.information_component(contingency_table)
            IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
            
            # Determine if the reaction meets criteria for a signal
            method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
            method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                
            if method_1 or method_2:
                #print(f"Signal found for {start_date}")
                first_found = start_date
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")
            else:
                #print("Signal not found")
                start_date = reduce_date_range_by_month(start_date)
                #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
            if start_date < end_date:
                break
        else:
            break
            
    if first_found == datetime(2000, 1, 1):
        print(f"Saved {symptom} as first_found = None because it was not detected by the end of an month")
        signals_dict_Novavax.append({"manufacturer": reaction["manufacturer"], "symptom": reaction["symptom"], "first_found": "NaN"})
    else:
            
        print(f"found month {first_found} for symptom {i} {symptom}, checking day")
        start_date = first_found
        while True:
            #print(f"checking day {start_date}")
                    #print(f"checking symptom {reaction['symptom']}")
            N_query = {"RECVDATE": {"$lte": start_date}}
            N = col.count_documents(N_query)
            
            D_query = {
                "RECVDATE": {"$lte": start_date},
                "vax_data": {
                    "$elemMatch": {
                        "VAX_MANU": reaction["manufacturer"],
                        "VAX_TYPE": reaction["vaccine"]
                    }
                }
            }
            D = col.count_documents(D_query)
            
            E_query = {
                "RECVDATE": {"$lte": start_date},
                "symptoms": {
                    "$elemMatch": {
                        "$in": [reaction["symptom"]]
                    }
                }
            }
            E = col.count_documents(E_query)
            
            DE_query = {'$and': [D_query, E_query]}
            DE = col.count_documents(DE_query)
            if DE >= 3: 
            
                De = D - DE
                dE = E - DE
                de = N - (DE + De + dE)
                
                    
                # Ensure all necessary data is available
                if None in [DE, dE, De, de]:
                    continue
                
                contingency_table = [
                    [DE, dE],
                    [De, de]
                ]
                #print(contingency_table)
                # Compute statistical measures
                rrr, sd_rrr = disproportionaly_analysis.relative_reporting_ratio(contingency_table), disproportionaly_analysis.sd_rrr(contingency_table)
                rrr_ci_lower, rrr_ci_upper = disproportionaly_analysis.confidence_interval(rrr, sd_rrr)
                
                prr, sd_prr = disproportionaly_analysis.proportional_reporting_ratio(contingency_table), disproportionaly_analysis.sd_prr(contingency_table)
                prr_ci_lower, prr_ci_upper = disproportionaly_analysis.confidence_interval(prr, sd_prr)
                
                ror, sd_ror = disproportionaly_analysis.reporting_odds_ratio(contingency_table), disproportionaly_analysis.sd_ror(contingency_table)
                ror_ci_lower, ror_ci_upper = disproportionaly_analysis.confidence_interval(ror, sd_ror)
                
                chi_square = disproportionaly_analysis.chi_square_yates(contingency_table)
                IC = disproportionaly_analysis.information_component(contingency_table)
                IC_ci_lower, IC_ci_upper = disproportionaly_analysis.confidence_interval_information_component(contingency_table, IC)
                
                # Determine if the reaction meets criteria for a signal
                method_1 = rrr > 2 and prr > 2 and ror > 2 and chi_square > 3.841
                method_2 = IC_ci_lower > 0 and ror_ci_lower > 1 and prr_ci_lower > 1 and rrr_ci_lower > 1
                    
                if method_1 or method_2:
                    #print(f"Signal found for {start_date}")
                    first_found = start_date
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Novavax.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop        
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")
                else:
                    #print("Signal not found")
                    if start_date.day == 1:  # Check if the day is the first day of the month
                        signal_info = {
                            "manufacturer": reaction["manufacturer"],
                            "symptom": reaction["symptom"],
                            "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                        }
                        signals_dict_Novavax.append(signal_info)
                        print(f"signal {i} {symptom} found at {first_found}")
                        break  # If it is the first day, exit the loop
                    start_date = reduce_date_range_by_day(start_date)
                    #print(f"Reducing by a month..., checking {start_date}")# Exit the loop if no signal is found anymore
    
            else: 
                #print(f"DE < 3 for {start_date}")
                signal_info = {
                    "manufacturer": reaction["manufacturer"],
                    "symptom": reaction["symptom"],
                    "first_found": first_found.strftime("%Y-%m-%d")  # Convert datetime to string
                }
                signals_dict_Novavax.append(signal_info)
                print(f"signal {i} {symptom} found at {first_found}")
                break
    

  4%|▍         | 1/23 [00:34<12:42, 34.66s/it]

Saved Headache as first_found = None because it was not detected by the end of an month


  9%|▊         | 2/23 [01:02<10:45, 30.72s/it]

Saved Nausea as first_found = None because it was not detected by the end of an month


 13%|█▎        | 3/23 [01:25<09:05, 27.29s/it]

...

Saved Erythema as first_found = None because it was not detected by the end of an month


 96%|█████████▌| 22/23 [09:10<00:18, 18.21s/it]

Saved Pruritus as first_found = None because it was not detected by the end of an month


100%|██████████| 23/23 [09:23<00:00, 24.50s/it]

Saved Urticaria as first_found = None because it was not detected by the end of an month


In [142]:
file_name_filtered = "First_found_filtered_Signals_Novavax.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_dict_Novavax, f)

In [128]:
signals_dict_Novavax

[{'manufacturer': 'NOVAVAX',
  'symptom': 'Paraesthesia',
  'first_found': '2022-08-07'},
 {'manufacturer': 'NOVAVAX',
  'symptom': 'Hypoaesthesia',
  'first_found': '2022-08-07'},
 {'manufacturer': 'NOVAVAX',
  'symptom': 'Pericarditis',
  'first_found': '2022-08-31'}]

### combine all signals and save as a dict with the symptom as key

In [157]:
signals_dict_combined = {}

# Merge signals_dict_Novavax
for entry in signals_dict_Novavax:
    symptom = entry.pop('symptom')  # Remove symptom from the entry and store it
    if symptom not in signals_dict_combined:
        signals_dict_combined[symptom] = [entry]
    else:
        signals_dict_combined[symptom].append(entry)

# Merge signals_dict_Janssen
for entry in signals_dict_Janssen:
    symptom = entry.pop('symptom')  # Remove symptom from the entry and store it
    if symptom not in signals_dict_combined:
        signals_dict_combined[symptom] = [entry]
    else:
        signals_dict_combined[symptom].append(entry)

# Merge signals_dict_Moderna
for entry in signals_dict_Moderna:
    symptom = entry.pop('symptom')  # Remove symptom from the entry and store it
    if symptom not in signals_dict_combined:
        signals_dict_combined[symptom] = [entry]
    else:
        signals_dict_combined[symptom].append(entry)

# Merge signals_dict_Pfizer
for entry in signals_dict_Pfizer:
    symptom = entry.pop('symptom')  # Remove symptom from the entry and store it
    if symptom not in signals_dict_combined:
        signals_dict_combined[symptom] = [entry]
    else:
        signals_dict_combined[symptom].append(entry)
        
print(signals_dict_combined)

{'Headache': [{'manufacturer': 'NOVAVAX', 'first_found': 'NaN'}, {'manufacturer': 'JANSSEN', 'first_found': '2021-03-04'}, {'manufacturer': 'MODERNA', 'first_found': '2020-12-21'}, {'manufacturer': 'PFIZER\\BIONTECH', 'first_found': '2020-12-16'}], 'Nausea': [{'manufacturer': 'NOVAVAX', 'first_found': 'NaN'}, {'manufacturer': 'JANSSEN', 'first_found': '2021-03-04'}, {'manufacturer': 'MODERNA', 'first_found': '2020-12-23'}, {'manufacturer': 'PFIZER\\BIONTECH', 'first_found': '2020-12-16'}], 'Vomiting': [{'manufacturer': 'NOVAVAX', 'first_found': 'NaN'}, {'manufacturer': 'JANSSEN', 'first_found': '2021-03-10'}, {'manufacturer': 'MODERNA', 'first_found': '2020-12-24'}, {'manufacturer': 'PFIZER\\BIONTECH', 'first_found': '2021-01-15'}], 'Myalgia': [{'manufacturer': 'NOVAVAX', 'first_found': 'NaN'}, {'manufacturer': 'JANSSEN', 'first_found': '2021-03-06'}, {'manufacturer': 'MODERNA', 'first_found': '2020-12-23'}, {'manufacturer': 'PFIZER\\BIONTECH', 'first_found': '2020-12-17'}], 'Arthralgi

In [158]:
file_name_filtered = "All_signals_filtered_first_found.json"
with open(file_name_filtered, 'w') as f:
    json.dump(signals_dict_combined, f)